In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.optimize import minimize
import yfinance as yfin
from statsmodels.tsa.regime_switching.markov_autoregression import MarkovAutoregression

In [2]:
nvda_df = yfin.download('NVDA',start = '1990-01-01', end = '2024-07-01')['Adj Close'].pct_change().dropna()
nvda_df.head()

[*********************100%%**********************]  1 of 1 completed


Date
1999-01-25    0.104764
1999-01-26   -0.077594
1999-01-27   -0.003110
1999-01-28   -0.003144
1999-01-29   -0.047019
Name: Adj Close, dtype: float64

In [11]:
#Markov AR(1)
model = MarkovAutoregression(nvda_df.values, k_regimes=2, order=1, switching_ar=True)
result = model.fit()

result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                         Markov Switching Model Results                         
================================================================================
Dep. Variable:                        y   No. Observations:                 6398
Model:             MarkovAutoregression   Log Likelihood               11976.735
Date:                  Sat, 27 Jul 2024   AIC                         -23939.470
Time:                          20:40:36   BIC                         -23892.124
Sample:                               0   HQIC                        -23923.081
                                 - 6398                                         
Covariance Type:                 approx                                         
                             Regime 0 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0044      0.001      4.050      0.000       0.002       0.007
ar.L1         -0.3736      0.050     -7.453      0.000      -0.472      -0.275
                             Regime 1 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001      0.001      0.147      0.883      -0.002       0.002
ar.L1          0.2924      0.042      6.951      0.000       0.210       0.375
                           Non-switching parameters                           
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2         0.0013   2.47e-05     51.569      0.000       0.001       0.001
                         Regime transition parameters                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p[0->0]        0.3502      0.080      4.379      0.000       0.193       0.507
p[1->0]        0.4998      0.088      5.704      0.000       0.328       0.672
==============================================================================

Warnings:
[1] Covariance matrix calculated using numerical (complex-step) differentiation.
"""

### Regime 0: (Mean Reverting Period)
The intercept and AR term are both signficant with the inclusion that the AR has a negative coefficien. This indicates that in this regime, the stock returns follow inversely from prior time period returns, demonstrating a period of high mean reversion. 

### Regime 1: (Momentum Period)
In this regime, only the AR Term is signficant with a positive coefficient, this would indicate a state of momentum or overall persistence of returns from a prior period.

### Non-Switching Variance:
Variance of the error term stays constant irregardless of state.

### Probabilities of State Switch:
We can form the stochastic matrix of the probabilities using the coefficients provided. The conditional probability of regime staying in state 0 given it is in state 0 is around 35.02%. While the switch to state 1 given state 0 is 49.98%. We can form the transition matrix using these figures:


In [26]:
transition_matrix = np.array([[.3502, 1-.3502],[.4998, 1-.4998]])
print(transition_matrix)

[[0.3502 0.6498]
 [0.4998 0.5002]]
